In [44]:
import os
import pandas
import pandas as pd
from oiss.session import Session
from oiss.ontology import IdentityTaskTemplate, TransformationTaskTemplate, OutputTaskTemplate
from oiss.ontologies.entity import IdentityStoragePattern, OutputStoragePattern
from oiss.ontologies.aic_entity import TransformationStoragePattern
from oiss.ontology import IdentityStorageTemplate, TransformationStorageTemplate, OutputStorageTemplate
from oiss.ontology import ProcessTemplate
from oiss.ontology import UBLTemplate
from io import BytesIO, StringIO
from rdflib import Graph
from collections import defaultdict
from oiss.ontology import StatementField  
from oiss.ontology import MemberDescriptionPattern  

## Use this code to checkout the contents of the specific csv file that was read into OISS in this process

In [45]:
# csvFile = 'https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d1950_c20250520.csv.gz'
# s3_bucket_name = 'nccf-sandbox-archive-workspace-us-east-1-188772587448' 
# file_key = 'foldername/SED.csv'

# def extract_csv(s3_bucket_name):

#     s3_client = boto3.client('s3')
    
#     file_key = 'foldername/SED.csv'

#     response = s3_client.get_object(Bucket=s3_bucket_name, Key=file_key)

#     csv_content = response['Body'].read().decode('utf-8')

#     df = pd.read_csv(StringIO(csv_content))
    
#     json_str = df.to_json(orient='records', indent=2) 

#     # print(json_str)

#     return df

# extract_csv(s3_bucket_name)

## Here is the code used in to create the two sed lambda functions

In [46]:
# ### Lambda function #1 (save csv to s3)

# import json
# import requests
# import io
# import pandas as pd
# import boto3

# def lambda_handler(event, context):

#     s3 = boto3.client('s3')

#     # Extract UBL-mapped inputs
#     csvFile = event["input_csv_s3_uri"]     # UBL input: input_csv_s3_uri
#     s3_bucket = event["output_s3_bucket"]   # UBL input: output_s3_bucket
#     s3_key = event.get("output_s3_key", "processed.csv")  # UBL input: output_s3_key (default fallback)

#     df = pd.read_csv(csvFile)

#     # Write processed CSV to S3
#     csv_buffer = io.StringIO()
#     df.to_csv(csv_buffer, index=False)
#     s3.put_object(Bucket=s3_bucket, Key=s3_key, Body=csv_buffer.getvalue())

#     # Return outputs mapped as UBL would expect
#     return {
#         "output_csv_s3_uri": f"s3://{s3_bucket}/{s3_key}"
        # }

In [47]:
### Lambda function #2 (retrieve csv file from s3 and load as a json)

# import io
# import pandas as pd
# import boto3

# def lambda_handler(event, context):

#     s3_bucket_name = event['s3_bucket_name']
    
#     file_key = event['file_key']

#     s3_client = boto3.client('s3')
    
#     response = s3_client.get_object(Bucket=s3_bucket_name, Key=file_key)

#     csv_content = response['Body'].read().decode('utf-8')

#     df = pd.read_csv(io.StringIO(csv_content))

#     json_str = df.to_json(orient='records', indent=2)

#     return {
#         "statusCode": 200,
#         "body": json_str
#     }

In [48]:
PROCESS_UUID = "lapenta25_process_template"
SED_AIU_UUID = "sed_aiu_template"
SED_AIC_UUID = "sed_aic_template"
SED_DIP_UUID = "sed_dip_template"

SED_SAVE_ARN = "arn:aws:lambda:us-east-1:188772587448:function:sed-test-lambda"
SED_READ_ARN = "arn:aws:lambda:us-east-1:188772587448:function:sed-test-lambda2"

AIU_TASK_ARN = "arn:aws:states:us-east-1:560271376700:stateMachine:archive-main-aiu-workflow"
AIU_UBL_ARN = "arn:aws:states:us-east-1:560271376700:stateMachine:archive-main-archive-data"

ARCHIVE_FILE_ARN = "arn:aws:lambda:us-east-1:560271376700:function:archive-main-archive-data-ArchiveFile"
CREATE_UUID_ARN = "arn:aws:lambda:us-east-1:560271376700:function:archive-main-archive-data-CreateUUID"

AIC_TASK_ARN = "arn:aws:states:us-east-1:560271376700:stateMachine:archive-main-aic-workflow"
AIC_UBL_ARN = "arn:aws:states:us-east-1:560271376700:stateMachine:archive-main-compute-aic-membership"
COMPUTE_AIC_MEMBER_ARN = "arn:aws:lambda:us-east-1:560271376700:function:archive-main-compute-aic-membership-ComputeAICMemberVariables"

DIP_TASK_ARN = "arn:aws:states:us-east-1:560271376700:stateMachine:archive-main-dip-workflow"
DIP_TRANSFORM_ARN = "arn:aws:states:us-east-1:188772587448:stateMachine:api-e2e-transform-data-osim"

PROCESS_ACCESS_PAYLOAD_ARN = "arn:aws:lambda:us-east-1:560271376700:function:archive-main-transform-data-osim-ProcessAccessPayload"
DIP_DELIVER_ARN = "arn:aws:states:us-east-1:560271376700:stateMachine:archive-main-deliver-data-swpc"
WRITE_RECEIPT_ARN = "arn:aws:lambda:us-east-1:560271376700:function:archive-main-deliver-data-osim-swpc-WriteReceipt"

In [49]:
%env OISS_PREFIX=sandbox
%env SPARQL_HOST=https://nccf-dev-sandbox-neptune.cluster-cd6wdbu8nvt0.us-east-1.neptune.amazonaws.com
%env SPARQL_PORT=8182
%env NEPTUNE_READ_ENDPOINT=https://nccf-dev-sandbox-neptune.cluster-cd6wdbu8nvt0.us-east-1.neptune.amazonaws.com #nccf-dev-sandbox-neptune.cluster-ro-cd6wdbu8nvt0.us-east-1.neptune.amazonaws.com
%env AWS_REGION=us-east-1
%env RDF_BUCKET_NAME=nccf-sandbox-archive-triplestore-us-east-1-188772587448
%env DEPLOY_BUCKET=nccf-sandbox-archive-workspace-us-east-1-188772587448
%env LD_LIBRARY_PATH=/usr/local/lib
%env ARC_CONFIG_FILE=my_install.yml

#Check if env is set
%env

env: OISS_PREFIX=sandbox
env: SPARQL_HOST=https://nccf-dev-sandbox-neptune.cluster-cd6wdbu8nvt0.us-east-1.neptune.amazonaws.com
env: SPARQL_PORT=8182
env: NEPTUNE_READ_ENDPOINT=https://nccf-dev-sandbox-neptune.cluster-cd6wdbu8nvt0.us-east-1.neptune.amazonaws.com #nccf-dev-sandbox-neptune.cluster-ro-cd6wdbu8nvt0.us-east-1.neptune.amazonaws.com
env: AWS_REGION=us-east-1
env: RDF_BUCKET_NAME=nccf-sandbox-archive-triplestore-us-east-1-188772587448
env: DEPLOY_BUCKET=nccf-sandbox-archive-workspace-us-east-1-188772587448
env: LD_LIBRARY_PATH=/usr/local/lib
env: ARC_CONFIG_FILE=my_install.yml


{'SHELL': '/bin/bash',
 'PWD': '/home/ec2-user',
 'LOGNAME': 'ec2-user',
 'XDG_SESSION_TYPE': 'tty',
 'MOTD_SHOWN': 'pam',
 'HOME': '/home/ec2-user',
 'SSL_CERT_DIR': '/etc/pki/tls/certs',
 'VSCODE_AGENT_FOLDER': '/home/ec2-user/.vscode-server',
 'SSH_CONNECTION': '127.0.0.1 45330 127.0.0.1 22',
 'XDG_SESSION_CLASS': 'user',
 'SELINUX_ROLE_REQUESTED': '',
 'USER': 'ec2-user',
 'SELINUX_USE_CURRENT_RANGE': '',
 'SHLVL': '1',
 'XDG_SESSION_ID': '1',
 'XDG_RUNTIME_DIR': '/run/user/1000',
 'SSL_CERT_FILE': '/etc/pki/ca-trust/extracted/pem/tls-ca-bundle.pem',
 'SSH_CLIENT': '127.0.0.1 45330 22',
 'VSCODE_CLI_REQUIRE_TOKEN': '<hidden>',
 'PATH': '/home/ec2-user/OISS/storm-events-graph/.sedvenv/bin:/home/ec2-user/.vscode-server/cli/servers/Stable-488a1f239235055e34e673291fb8d8c810886f81/server/bin/remote-cli:/home/ec2-user/bin:/home/ec2-user/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/ec2-user/.local/bin:/home/ec2-user/bin:/usr/local/bin:/usr/local/nodejs/current/bin:/home/ec2

# First: Initialize process template

In [50]:
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"
api_session = Session(default_session_template='sandbox-ephemeral-session')
process_template = ProcessTemplate(api_session, purpose="Storm Events Database Modernization", utility="Storm Event Data Modeling", node_id=PROCESS_UUID)
process_template.set_title("Storm Events Database Model")

'Storm Events Database Model'

# **Archival Information Unit**

### Step 1: Create AIU Storage Pattern

In [51]:
aiu_storage_pattern = IdentityStoragePattern(api_session, title='Storm Events Database AIU Storage Pattern')

### Add Preservation Information


In [52]:
packaging = aiu_storage_pattern.add_packaging("Packaging")
packaging_semantic = packaging.add_semantic_representation("packing semantic")
packaging_structure = packaging.add_structure_representation("packing structure")
packaging_semantic.node.addsMeaningTo.append(packaging_structure.node)

description = aiu_storage_pattern.add_description("Storm Events Database Description") 
description_semantic = description.add_semantic_representation("Description")
description_structure = description.add_structure_representation("Format")

file_content = aiu_storage_pattern.set_content("File", field={"is_link":True}) 
file_content_semantic = file_content.add_semantic_representation("Description")
file_content_structure = file_content.add_structure_representation("Format")

checksum = aiu_storage_pattern.add_fixity("Checksum", field={'title': "Value", "is_link":False}) 
checksum_semantic = checksum.add_semantic_representation("Definition")
checksum_structure = checksum.add_structure_representation("Format")
checksum_semantic.node.addsMeaningTo.append(checksum_structure.node)

sed_reference = aiu_storage_pattern.add_reference("Website Reference", field={'title': "Value", "is_link":True, 'value':'https://www.ncei.noaa.gov/pub/data/swdi/stormevents/'})
ref_semantic = sed_reference.add_semantic_representation("Description")
ref_structure = sed_reference.add_structure_representation("Format")
ref_semantic.node.addsMeaningTo.append(ref_structure.node)

csv_reference = aiu_storage_pattern.add_reference("Data Reference", field={'title': "Value", "is_link":True, 'value':'https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/'})
ref_semantic = csv_reference.add_semantic_representation("Description")
ref_structure = csv_reference.add_structure_representation("Format")
ref_semantic.node.addsMeaningTo.append(ref_structure.node)

access_rights = aiu_storage_pattern.add_access_rights("License") 
access_rights_semantic = access_rights.add_semantic_representation("License semantic")
access_rights_structure = access_rights.add_structure_representation("License structure")

context = aiu_storage_pattern.add_context("Provider")
context_semantic = context.add_semantic_representation("context semantic")
context_structure = context.add_structure_representation("context structure")
context_semantic.node.addsMeaningTo.append(context_structure.node)

aiu_storage_pattern.visualize().show("AIU_storage_pattern.html")

AIU_storage_pattern.html


In [53]:
# validationreport = aiu_storage_pattern.validate()
# print(validationreport.message)

### Step 2: Create AIU Storage Template

In [54]:
aiu_storage_template = IdentityStorageTemplate(api_session, source_pattern=aiu_storage_pattern, node_id=SED_AIU_UUID)
aiu_storage_template.set_title("Storm Events Database AIU storage template")
# aiu_storage_template.deploy(overwrite=True)

# aiu_storage_template.validate()
# aiu_storage_template.get_fields()
aiu_storage_template.visualize().show("AIU_storage_template.html")

AIU_storage_template.html


### Step 4: Create AIU Task Template

In [55]:
aiu_task_template = IdentityTaskTemplate(api_session, purpose="example", utility="Store Storm Events Database CSV file in s3 Bucket")
aiu_task_template.set_target(aiu_storage_template)
aiu_task_template.set_title('Storm Events Database AIU Task Template')
process_template.set_aiu_task(aiu_task_template)

# aiu_task_template.validate()
aiu_task_template.get_fields()
aiu_task_template.visualize().show("AIU_task_template.html")

AIU_task_template.html


### Step 5: Create UBL Template (Save SED CSV to s3 bucket)

In [56]:
aiu_ubl_template = UBLTemplate(api_session, namespace=aiu_task_template.namespace)
aiu_task_template.set_ubl(aiu_ubl_template)
aiu_ubl_template.set_packaging_field(packaging="Resource Identifier", field="Value", value=SED_SAVE_ARN) 
aiu_ubl_template.get_fields()

{'runtime_fields': {'placeholder1': <oiss.ontologies.fmk.LiteralField at 0x7fb3e5478e50>,
  'placeholder2': <oiss.ontologies.fmk.LiteralField at 0x7fb3e5478fa0>},
 'resource_identifier': {'value': <oiss.ontologies.fmk.LiteralField at 0x7fb3e54785e0>},
 'description': {'description_value': <oiss.ontologies.fmk.LiteralField at 0x7fb3e5478b50>},
 'access_rights': {'access_rights_value': <oiss.ontologies.fmk.LiteralField at 0x7fb3e54787f0>},
 'context': {'context_value': <oiss.ontologies.fmk.LiteralField at 0x7fb3e546b370>},
 'fixity': {'fixity_value': <oiss.ontologies.fmk.LiteralField at 0x7fb3e546bf10>},
 'provenance': {'provenance_value': <oiss.ontologies.fmk.LiteralField at 0x7fb3e546bbb0>},
 'reference': {'reference_value': <oiss.ontologies.fmk.LiteralField at 0x7fb3e546b700>}}

### Step 6: Create UBL input and output fields:

In [57]:
aiu_ubl_template.add_input("input_csv_s3_uri", is_required=True)  
aiu_ubl_template.add_input("output_s3_bucket", is_required=True) 
aiu_ubl_template.add_input("output_s3_key")

aiu_ubl_template.add_output("s3_uri")
aiu_ubl_template.add_output("checksum")
aiu_ubl_template.add_output("website_reference")
aiu_ubl_template.add_output("data_reference")
aiu_ubl_template.add_output("license")
aiu_ubl_template.add_output("provider")

aiu_ubl_template.visualize().show("AIU_ubl_template.html")

AIU_ubl_template.html


### Step 7: Map AIU UBL output fields to AIU storage template fields

In [58]:
aiu_task_template.match_field(aiu_ubl_template.outputs.s3_uri, aiu_storage_template.file.file_link)
aiu_task_template.match_field(aiu_ubl_template.outputs.checksum, aiu_storage_template.checksum.value)
aiu_task_template.match_field(aiu_ubl_template.outputs.website_reference, aiu_storage_template.website_reference.value)
aiu_task_template.match_field(aiu_ubl_template.outputs.data_reference, aiu_storage_template.data_reference.value)
aiu_task_template.match_field(aiu_ubl_template.outputs.license, aiu_storage_template.license.license_value)
aiu_task_template.match_field(aiu_ubl_template.outputs.provider, aiu_storage_template.provider.provider_value)

# **Archival Information Context**

### Step 1: Create AIC Storage Pattern

In [59]:
aic_storage_pattern = TransformationStoragePattern(api_session, "Storm Events Database AIC Storage Pattern", labels=["Storms", "Extreme Weather", "Hurricane", 
                                                    "Natural Disaster", "Cyclone", "Monsoon", "Typhoon", "Cyclone", "Supercell"])

### Add AIC Preservation Information

In [60]:
### Add preservation information

reference = aic_storage_pattern.add_reference('SED DOI', field={'title':'Value', 'is_link': False}, labels=["DOI"])
semantic = reference.add_semantic_representation("Description")
structure = reference.add_structure_representation("Format")
semantic.node.addsMeaningTo.append(structure.node)

context = aic_storage_pattern.add_context("SED Help Guide", labels=["guidance"])
context.add_semantic_representation("Meaning")
context.add_structure_representation("Format")

data_credits = aic_storage_pattern.add_context("SED Data Credits", labels=["data-rights"]) ### <---------- new
data_credits.add_semantic_representation("Meaning")
data_credits.add_structure_representation("Format")

author_credits = aic_storage_pattern.add_context("SED Author Credits", labels=["contributor"]) ### <---------- new
author_credits.add_semantic_representation("Meaning")
author_credits.add_structure_representation("Format")

provenance = aic_storage_pattern.add_provenance("SED Provenance", labels=["provenance"])
provenance.add_semantic_representation("Meaning")
provenance.add_structure_representation("Format")

packaging = aic_storage_pattern.add_packaging("SED Packaging", labels=["packaging"])
packaging.add_semantic_representation("Packaging Strategy")
packaging.add_structure_representation("Packaging Format")
packaging.node.identifiesPackage.append(aic_storage_pattern.node)

description = aic_storage_pattern.add_overview_description("SED Overview Description", labels=["sed-overview"])
description.add_semantic_representation("Meaning")
description.add_structure_representation("Format")

access = aic_storage_pattern.add_access_rights("SED Access Rights", labels=["sed-access-rights"])
access.add_semantic_representation("Meaning")
access.add_structure_representation("Format") 

checksum = aic_storage_pattern.add_fixity("SED Checksum", field={'title':'Value', 'is_link':False}, labels=['sed-checksum'])
checksum_semantic = checksum.add_semantic_representation("Definition")
checksum_structure = checksum.add_structure_representation("Format")
checksum_semantic.node.addsMeaningTo.append(checksum_structure.node)

file_content = aic_storage_pattern.set_content("SED Members", field={'is_link': True, 'value':'https://www.ncdc.noaa.gov/stormevents/'}, labels=['sed-members'])
file_content.add_value_field("Pattern Placeholder")
file_semantic = file_content.add_semantic_representation("SED Description")
file_structure = file_content.add_structure_representation("Format")
file_semantic.node.addsMeaningTo.append(file_structure.node)

### Add SED member description


In [61]:
main_sed_graph = aic_storage_pattern.add_member_description("storm_events_database_member_description")
structure = main_sed_graph.add_structure_representation("Description") 
semantic = main_sed_graph.add_semantic_representation("Format")
semantic.node.addsMeaningTo.append(structure.node)
main_sed_graph.node.derivedFromInformationPackage.append(aic_storage_pattern.node)

### Initialize Object Entities: Storm Event

In [62]:
storm_event = main_sed_graph.add_object("storm_event", namespace="http://www.w3.org/2001/XMLSchema#", namespace_class='location')

storm_event_name = storm_event.add_literal("name", labels=["name"])
storm_event.connect(storm_event_name, title="hasName", namespace="https://schema.org/", namespace_class = "name")

storm_event_type = storm_event.add_literal("type", labels=["type"])
storm_event.connect(storm_event_type, title="hasType", namespace="https://schema.org/", namespace_class = "type")

storm_event_magnitude = storm_event.add_literal("magnitude", labels=["magnitude"])
storm_event.connect(storm_event_magnitude, title="hasMagnitude", namespace="https://schema.org/", namespace_class = "magnitude")

storm_event_id = storm_event.add_literal('event_id', labels=["id"])
storm_event.connect(storm_event_id, title="hasId", namespace="https://schema.org/", namespace_class="id")

property_damage = storm_event.add_literal('property_damage', namespace='http://www.w3.org/2001/XMLSchema#', namespace_class="int")
storm_event.connect(property_damage, title="hasDamage", namespace="https://schema.org/", namespace_class="damage")

storm_event_state = storm_event.add_literal("state", labels=["state"])
storm_event.connect(storm_event_state, title='hasState', namespace="https://schema.org/", namespace_class="state")

storm_event_year = storm_event.add_literal("year", labels=["year"])
storm_event.connect(storm_event_year, title="hasYear", namespace="https://schema.org/", namespace_class="year")

storm_event_month = storm_event.add_literal("month_name", labels=["month"])
storm_event.connect(storm_event_month, title="hasMonth", namespace="https://schema.org/", namespace_class="month")

storm_event_narrative = storm_event.add_literal("event_narrative", labels=["narrative"])
storm_event.connect(storm_event_narrative, title="hasNarrative", namespace="https://schema.org/", namespace_class="narrative")

### Initialize Object Entities: WFO

In [63]:
weatherForecastOffice = main_sed_graph.add_object("weather_forecast_office", namespace="http://www.w3.org/2001/XMLSchema#", namespace_class='string')

office_name = weatherForecastOffice.add_literal("name", namespace_class="string")
weatherForecastOffice.connect(office_name, title="hasName", namespace="https://schema.org/", namespace_class="name")

office_class = weatherForecastOffice.add_literal("class", namespace_class="string")
weatherForecastOffice.connect(office_name, title="hasClass", namespace="https://schema.org/", namespace_class="class")

office_region = weatherForecastOffice.add_literal("region", namespace_class="string")
weatherForecastOffice.connect(office_name, title="hasRegion", namespace="https://schema.org/", namespace_class="region")

office_id = weatherForecastOffice.add_literal("office_id", namespace_class="string")
weatherForecastOffice.connect(office_id, title='hasId', namespace="https://schema.org/", namespace_class="id")

office_state = weatherForecastOffice.add_literal("state", labels=["state"])
weatherForecastOffice.connect(office_state, title='hasState', namespace="https://schema.org/", namespace_class="state")

office_statefips = weatherForecastOffice.add_literal("state_fips", labels=["state fips"])
weatherForecastOffice.connect(office_statefips, title='hasFips', namespace="https://schema.org/", namespace_class="state fips")

office_timezone = weatherForecastOffice.add_literal("timezone", labels=["timezone"])
weatherForecastOffice.connect(office_timezone, title='hasTimezone', namespace="https://schema.org/", namespace_class="timezone")

### Initialize Object Entities: Storm_episode

In [64]:
storm_episode = main_sed_graph.add_object("Storm Episode", namespace="http://www.w3.org/2001/XMLSchema#", namespace_class='Location') 

storm_episode_name = storm_episode.add_literal("Episode name")
storm_episode.connect(storm_episode_name, title="hasName", namespace="https://schema.org/", namespace_class="name")

storm_episode_type = storm_episode.add_literal("Episode type", namespace_class="string")
storm_episode.connect(storm_episode_type, title="hasType", namespace="https://schema.org/", namespace_class="type")

storm_episode_magnitude = storm_episode.add_literal("magnitude", namespace_class="int")
storm_episode.connect(storm_episode_magnitude, title="hasMagnitude", namespace="https://schema.org/", namespace_class="magnitude")

### Initialize Object Entities: Location

In [65]:
location = main_sed_graph.add_object("Location", namespace="http://www.w3.org/2001/XMLSchema#", namespace_class='Location')

location_name = location.add_literal("name", namespace_class="string")
location.connect(location_name, title="hasName", namespace="https://schema.org/", namespace_class="name")

location_type = location.add_literal("type", namespace_class="string") 
location.connect(location_type, title="hasType", namespace="https://schema.org/", namespace_class="type")

location_lat = location.add_literal("Latitude", namespace_class='int')
location.connect(location_lat, title="hasLat", namespace="https://schema.org/", namespace_class="latitude")

location_lon = location.add_literal("Longitude", namespace_class='int')
location.connect(location_lon, title="hasLon", namespace="https://schema.org/", namespace_class="longitude")

location_county = location.add_literal("County", namespace_class='int')
location.connect(location_county, title="hasCounty", namespace="https://schema.org/", namespace_class="county")

### Initialize Object Entities: Socioeconomic impact

In [66]:
impact = main_sed_graph.add_object("Socioeconomic Impact", namespace="http://www.w3.org/2001/XMLSchema#", namespace_class='Location')

impact_type = impact.add_literal("type", namespace_class="string") 
impact.connect(impact_type, title="hasType", namespace="https://schema.org/", namespace_class="type")

impact_casualties = impact.add_literal("casualties", namespace_class="int") 
impact.connect(impact_casualties, title="hasCasualties", namespace="https://schema.org/", namespace_class="casualties")

impact_economic = impact.add_literal("economic", namespace_class="int") 
impact.connect(impact_economic, title="hasEconomic", namespace="https://schema.org/", namespace_class="economic")

In [67]:
storm_event.connect(weatherForecastOffice, title="hasForecastOffice", namespace="https://schema.org/", namespace_class="office")
storm_event.connect(impact, title="hasDamage", namespace="https://schema.org/", namespace_class="damage")
storm_event.connect(location, title="hasLocation", namespace="https://schema.org/", namespace_class="location")
location.connect(impact, title="hasImpact", namespace="https://schema.org/", namespace_class="impact")

# aic_storage_pattern.visualize().show("Test_AIC_Storage_Pattern.html")
# main_sed_graph.visualize(notebook = False, hierarchical= True, isCDNRemote= True, uml=True).show("Test_AIC_memberdescription1.html", notebook = False)
# main_sed_graph.visualize().show("Test_AIC_memberdescription2.html")
# aic_storage_template.visualize().show("AIC_storage_pattern.html")

### Step 2: Create AIC Storage Template

In [68]:
aic_storage_template = TransformationStorageTemplate(api_session, source_pattern=aic_storage_pattern, node_id=SED_AIC_UUID)
aic_storage_template.set_title("Storm Events Database AIC Storage Template")
# aic_storage_template.deploy(overwrite=True)

# aiu_storage_template.validate()
aiu_storage_template.visualize().show("AIC_storage_template.html")

AIC_storage_template.html


## The purpose of the code below is to provide context to the architecture of our member description and its associated fields

In [69]:
aic_storage_pattern.member_descriptions

{<oiss.ontologies.aic_entity.MemberDescriptionPattern at 0x7fb3e548df10>}

In [70]:
print(aic_storage_pattern.get_title())

Storm Events Database AIC Storage Pattern


In [71]:
print(aic_storage_template.get_title())
aic_storage_template.information_objects

Storm Events Database AIC Storage Template


In [72]:
member_Desc = list(aic_storage_pattern.member_descriptions)[0]
# print(member_Desc.get_title())
fields = list(member_Desc.fields)
fields

In [73]:
from collections import defaultdict

populated_dictionary = {}
literal_dictionary = {}
statement_dictionary = {}
property_dictionary = {}
object_dictionary = {}

for field_a in fields:
    
    if True:

        populated_dictionary[field_a.get_title()] = field_a.__class__.__name__
        
        grouped = defaultdict(list)
        for k, v in populated_dictionary.items():
            grouped[v].append(k)

        max_length = max(len(keys) for keys in grouped.values())

        data = {}
        for value, keys in grouped.items():
            keys = keys + [None] * (max_length - len(keys))
            data[value] = keys

        df = pd.DataFrame(data)

    if field_a.__class__.__name__ == "LiteralField":

        literal_dictionary[field_a.get_title()] = field_a.__class__.__name__
        literal_df = pd.DataFrame.from_dict(literal_dictionary, orient='index', columns=['Field Type'])

    elif field_a.__class__.__name__ == "StatementField":

        statement_dictionary[field_a.get_title()] = field_a.__class__.__name__
        statement_df = pd.DataFrame.from_dict(statement_dictionary, orient='index', columns=['Field Type'])

    elif field_a.__class__.__name__ == "PropertyField":
        property_dictionary[field_a.get_title()] = field_a.__class__.__name__
        property_df = pd.DataFrame.from_dict(property_dictionary, orient='index', columns=['Field Type'])

    else:
        object_dictionary[field_a.get_title()] = field_a.__class__.__name__
        object_df = pd.DataFrame.from_dict(object_dictionary, orient='index', columns=['Field Type'])

In [74]:
df

,StatementField,LiteralField,PropertyField,ObjectField
0,Socioeconomic Impact hasType type,name,hasProperty,storm_event
1,storm_event hasProperty name,type,hasClass,weather_forecast_office
2,weather_forecast_office hasProperty name,class,hasType,Socioeconomic Impact
3,storm_event hasName name,magnitude,hasName,Location
4,storm_event hasProperty type,timezone,hasRegion,Storm Episode
5,weather_forecast_office hasFips state_fips,event_narrative,hasCasualties,None
6,weather_forecast_office hasName name,state_fips,hasTimezone,None
7,weather_forecast_office hasState state,state,hasNarrative,None
8,weather_forecast_office hasProperty state_fips,office_id,hasFips,None
9,storm_event hasMonth month_name,event_id,hasState,None


In [75]:
object_df

,Field Type
storm_event,ObjectField
weather_forecast_office,ObjectField
Socioeconomic Impact,ObjectField
Location,ObjectField
Storm Episode,ObjectField


In [76]:
literal_df

,Field Type
name,LiteralField
type,LiteralField
class,LiteralField
magnitude,LiteralField
timezone,LiteralField
event_narrative,LiteralField
state_fips,LiteralField
state,LiteralField
office_id,LiteralField
event_id,LiteralField


In [77]:
property_df

,Field Type
hasProperty,PropertyField
hasClass,PropertyField
hasType,PropertyField
hasName,PropertyField
hasRegion,PropertyField
hasCasualties,PropertyField
hasTimezone,PropertyField
hasNarrative,PropertyField
hasFips,PropertyField
hasState,PropertyField


In [78]:
statement_df

,Field Type
Socioeconomic Impact hasType type,StatementField
storm_event hasProperty name,StatementField
weather_forecast_office hasProperty name,StatementField
storm_event hasName name,StatementField
storm_event hasProperty type,StatementField
weather_forecast_office hasFips state_fips,StatementField
weather_forecast_office hasName name,StatementField
weather_forecast_office hasState state,StatementField
weather_forecast_office hasProperty state_fips,StatementField
storm_event hasMonth month_name,StatementField


## Here we are able to set the value of our object properties, but this is already done by OISS so we don't need to include it in our process

In [79]:
# aic_storage_template.storm_events_database_member_description.storm_event.name.set_value("New Storm")
# aic_storage_template.storm_events_database_member_description.storm_event.type.set_value("Thunderstorm")
# aic_storage_template.storm_events_database_member_description.storm_event.month_name.set_value("April")
# aic_storage_template.storm_events_database_member_description.storm_event.year.set_value("2025")
# aic_storage_template.storm_events_database_member_description.storm_event.state.set_value("North Carolina")
# aic_storage_template.storm_events_database_member_description.storm_event.property_damage.set_value("x thousand U.S. dollars")
# aic_storage_template.storm_events_database_member_description.storm_event.event_id.set_value("XXXX")
# aic_storage_template.storm_events_database_member_description.storm_event.magnitude.set_value("3")
# aic_storage_template.storm_events_database_member_description.storm_event.event_narrative.set_value("A new description")

### The code below is a work-in-progress member description filter, which is meant to filter the contents of our member description to --only-- visualize statement fields.

In [80]:
member_Desc = list(aic_storage_pattern.member_descriptions)[0]

fields = list(member_Desc.fields)

filtered_member_Desc = MemberDescriptionPattern(
    api_session,
    aic_pattern=aic_storage_pattern,  
    title="Filtered Statement Fields"
)

filtered_member_Desc.copy_from_member_description(member_Desc)

for field in list(filtered_member_Desc.fields):
    if field.__class__.__name__ != "StatementField":
        filtered_member_Desc.fields.remove(field)

# filtered_member_Desc.visualize().show("field_test.html")

### Step 3: Create AIC Task Template

In [81]:
aic_task_template = TransformationTaskTemplate(api_session, aic_template=aic_storage_template, utility="Read Storm Events Database CSV file from s3 Bucket")
aic_task_template.set_target(aic_storage_template)
aic_task_template.set_title('Storm Events Database AIC Extraction Task Template')
aic_task_template.set_packaging_field(packaging="Resource Identifier", field="Value", value=AIC_TASK_ARN)

# aic_task_template.validate()
# aic_task_template.get_fields()
aic_task_template.visualize().show("AIC_task_template.html")

AIC_task_template.html


### Step 4: Create AIC UBL Template (Read in CSV from s3 bucket and create JSON using OISS)

In [82]:
aic_ubl_template = UBLTemplate(api_session, namespace=aic_task_template.namespace)
aic_ubl_template.set_packaging_field(packaging="Resource Identifier", field="Value", value=SED_READ_ARN)
aic_task_template.set_ubl(aic_ubl_template)

# aic_ubl_template.get_fields()
aic_ubl_template.visualize().show("AIC_ubl_template.html")

AIC_ubl_template.html


### Step 5: Initialize AIC UBL input and output fields:

In [83]:
aic_ubl_template.add_input("Access Key")
aic_ubl_template.add_input("Access Bucket")
aic_ubl_template.add_input("Member Type", value="csv")
aic_ubl_template.add_input("Destination Prefix", value="")
aic_ubl_template.add_input("S3 URI")

storm_event_name = aic_ubl_template.add_output("storm_event_name")
storm_event_id = aic_ubl_template.add_output("storm_event_id")
storm_event_type = aic_ubl_template.add_output("storm_event_type")
storm_event_magnitude = aic_ubl_template.add_output("storm_event_magnitude")
storm_event_state = aic_ubl_template.add_output("storm_event_state")
storm_event_year = aic_ubl_template.add_output("storm_event_year")
storm_event_state_fips = aic_ubl_template.add_output("storm_event_state_fips")
storm_event_month_name = aic_ubl_template.add_output("storm_event_month_name")
storm_event_property_damage = aic_ubl_template.add_output("storm_event_property_damage")
storm_event_timezone = aic_ubl_template.add_output("storm_event_timezone")
storm_event_narrative = aic_ubl_template.add_output("storm_event_narrative")

### Step 6: Link AIC UBL output fields to the AIC storage template fields

In [84]:
aic_task_template.match_field(aic_ubl_template.outputs.storm_event_name, storm_event_name)
aic_task_template.match_field(aic_ubl_template.outputs.storm_event_id, storm_event_id)
aic_task_template.match_field(aic_ubl_template.outputs.storm_event_type, storm_event_type)
aic_task_template.match_field(aic_ubl_template.outputs.storm_event_magnitude, storm_event_magnitude)
aic_task_template.match_field(aic_ubl_template.outputs.storm_event_state, storm_event_state)
aic_task_template.match_field(aic_ubl_template.outputs.storm_event_year, storm_event_year)
aic_task_template.match_field(aic_ubl_template.outputs.storm_event_state_fips, storm_event_state_fips)
aic_task_template.match_field(aic_ubl_template.outputs.storm_event_month_name, storm_event_month_name)
aic_task_template.match_field(aic_ubl_template.outputs.storm_event_property_damage, storm_event_property_damage)
aic_task_template.match_field(aic_ubl_template.outputs.storm_event_timezone, storm_event_timezone)
aic_task_template.match_field(aic_ubl_template.outputs.storm_event_narrative, storm_event_narrative)

# **Dissemination Infromation Package**

### Step 1: Create DIP Storage Pattern

In [85]:
dip_storage_pattern = OutputStoragePattern(api_session, "Storm Events Database Delivery Record")
response = dip_storage_pattern.set_content("Message", field={"title":"Value"})
response.add_semantic_representation("Message Semantic Response")
response.add_structure_representation("Message Structure Response")

dip_storage_pattern.visualize().show("DIP_storage_template.html")

DIP_storage_template.html


### Add preservation information

In [86]:
req_val = dip_storage_pattern.set_content("Request value")
req_val.add_semantic_representation("Request Semantic Rep")
req_val.add_structure_representation("Request Structure Rep")

resp_val = dip_storage_pattern.set_content("Response value")
resp_val.add_semantic_representation("Response Semantic Rep")
resp_val.add_structure_representation("Response Structure Rep")

resp_message = dip_storage_pattern.set_content("Response Message")
resp_message.add_semantic_representation("Response Semantic Rep")
resp_message.add_structure_representation("Response Structure Rep")

req_time = dip_storage_pattern.set_content("Time of request")
req_time.add_semantic_representation("Time Req Definition")
req_time.add_structure_representation("Time Req Format")

resp_time = dip_storage_pattern.set_content("Time of response")
resp_time.add_semantic_representation("Time Response Semantic Rep")
resp_time.add_structure_representation("Time Response Structure Rep")

dip_storage_pattern.visualize().show("DIP_storage_pattern.html")

DIP_storage_pattern.html


### Step 2: Create DIP Storage Template 

In [87]:
dip_storage_template = OutputStorageTemplate(api_session, source_pattern= dip_storage_pattern, node_id=SED_DIP_UUID)
# dip_storage_template.deploy(overwrite=True)

# dip_storage_template.validate()
# dip_storage_template.get_fields()
dip_storage_template.visualize().show('DIP_storage_template.html')

DIP_storage_template.html


### Step 3: Create DIP Task Template

In [88]:
dip_task_template = OutputTaskTemplate(api_session, purpose='DIP AIC Task', utility="Test DIP Task template") 
dip_task_template.set_packaging_field(packaging="Resource Identifier", field="Value", value=DIP_TASK_ARN)
dip_task_template.set_target(dip_storage_template)

# dip_task_template.validate()
# dip_task_template.get_fields()
dip_task_template.visualize().show('DIP_task_template.html')

DIP_task_template.html


### Step 4: Create DIP UBL Template #1

In [89]:
dip_transform_ubl = UBLTemplate(api_session, namespace=dip_task_template.namespace)
dip_task_template.set_transform_data_ubl(dip_transform_ubl)
dip_transform_ubl.set_packaging_field(packaging="Resource Identifier", field="Value", value="")

dip_transform_ubl.visualize().show("DIP_transform_ubl.html")

DIP_transform_ubl.html


### Step 6: Create DIP UBL Template #2

In [90]:
dip_deliver_ubl = UBLTemplate(api_session, namespace=dip_task_template.namespace)
dip_task_template.set_deliver_data_ubl(dip_deliver_ubl)
dip_deliver_ubl.set_packaging_field(packaging="Resource Identifier", field="Value", value="")

dip_deliver_ubl.visualize().show("DIP_deliver_ubl.html")

DIP_deliver_ubl.html


### Initialize DIP input and output fields 

In [91]:
# dip_deliver_ubl.add_input("Default")

dip_deliver_ubl.add_input("Access Key")
dip_deliver_ubl.add_input("Access Bucket")
dip_deliver_ubl.add_input("Member Type", value="csv")
dip_deliver_ubl.add_input("Destination Prefix", value='')
dip_deliver_ubl.add_input("S3 URI")

### Construct output fields 

dip_deliver_ubl.add_output("request_value")
dip_deliver_ubl.add_output("response_value")
dip_deliver_ubl.add_output("delivery_success")
dip_deliver_ubl.add_output("time_of_request")
dip_deliver_ubl.add_output("time_of_response")

# dip_deliver_ubl.get_fields()
# dip_deliver_ubl.visualize().show('DIP_UBL_Template.html')

## Use the code below to understand what to map

In [92]:
aiu_ubl_template.outputs

namespace(s3_uri=<oiss.ontologies.fmk.LiteralField at 0x7fb3e550e760>,
          checksum=<oiss.ontologies.fmk.LiteralField at 0x7fb3e5c82c10>,
          website_reference=<oiss.ontologies.fmk.LiteralField at 0x7fb3e550eac0>,
          data_reference=<oiss.ontologies.fmk.LiteralField at 0x7fb3e550e3d0>,
          license=<oiss.ontologies.fmk.LiteralField at 0x7fb3e5467e80>,
          provider=<oiss.ontologies.fmk.LiteralField at 0x7fb3e5467e20>)

In [93]:
aiu_ubl_template.inputs

namespace(input_csv_s3_uri=<oiss.ontologies.fmk.LiteralField at 0x7fb3e54b9e20>,
          output_s3_bucket=<oiss.ontologies.fmk.LiteralField at 0x7fb3e54b9cd0>,
          output_s3_key=<oiss.ontologies.fmk.LiteralField at 0x7fb3e54782e0>)

In [94]:
dip_deliver_ubl.inputs

namespace(access_key=<oiss.ontologies.fmk.LiteralField at 0x7fb3e544c460>,
          access_bucket=<oiss.ontologies.fmk.LiteralField at 0x7fb3e543c640>,
          member_type=<oiss.ontologies.fmk.LiteralField at 0x7fb3e5404190>,
          destination_prefix=<oiss.ontologies.fmk.LiteralField at 0x7fb3e544c340>,
          s3_uri=<oiss.ontologies.fmk.LiteralField at 0x7fb3e544c310>)

In [95]:
dip_deliver_ubl.outputs

namespace(request_value=<oiss.ontologies.fmk.LiteralField at 0x7fb3e544c760>,
          response_value=<oiss.ontologies.fmk.LiteralField at 0x7fb3e544c280>,
          delivery_success=<oiss.ontologies.fmk.LiteralField at 0x7fb3e544c3a0>,
          time_of_request=<oiss.ontologies.fmk.LiteralField at 0x7fb3e544c670>,
          time_of_response=<oiss.ontologies.fmk.LiteralField at 0x7fb3e544c0d0>)

In [96]:
dip_storage_template.get_fields()

{'message': {'value': <oiss.ontologies.fmk.LiteralField at 0x7fb3e51df4c0>},
 'request_value': {'request_value_value': <oiss.ontologies.fmk.LiteralField at 0x7fb3e51df5b0>},
 'response_value': {'response_value_value': <oiss.ontologies.fmk.LiteralField at 0x7fb3e5727f70>},
 'response_message': {'response_message_value': <oiss.ontologies.fmk.LiteralField at 0x7fb3e51e3220>},
 'time_of_request': {'time_of_request_value': <oiss.ontologies.fmk.LiteralField at 0x7fb3e51e3940>},
 'time_of_response': {'time_of_response_value': <oiss.ontologies.fmk.LiteralField at 0x7fb3e51e3cd0>}}

In [97]:
dip_task_template.match_field(aiu_ubl_template.outputs.s3_uri, dip_deliver_ubl.inputs.s3_uri)
dip_task_template.match_field(aiu_ubl_template.inputs.output_s3_bucket, dip_deliver_ubl.inputs.access_bucket)
dip_task_template.match_field(aiu_ubl_template.inputs.output_s3_key, dip_deliver_ubl.inputs.access_key)

### Link DIP UBL output fields to DIP storage template fields

In [98]:
dip_task_template.match_field(dip_deliver_ubl.outputs.request_value, dip_storage_template.request_value.request_value_value)
dip_task_template.match_field(dip_deliver_ubl.outputs.response_value, dip_storage_template.response_value.response_value_value)
dip_task_template.match_field(dip_deliver_ubl.outputs.delivery_success, dip_storage_template.response_message.response_message_value)
dip_task_template.match_field(dip_deliver_ubl.outputs.time_of_request, dip_storage_template.time_of_request.time_of_request_value)
dip_task_template.match_field(dip_deliver_ubl.outputs.time_of_response, dip_storage_template.time_of_response.time_of_response_value)

# **Initialize OISS Process**

### Step 1: Link the AIU Task to the AIC Task

In [99]:
process_template.link_aic_task(aiu_task_template, aic_task_template)

### Step 2: Link the AIU Task to the DIP Task

In [100]:
process_template.link_dip_task(aiu_task_template, dip_task_template)

### Step 5: Deploy the process

In [101]:
process_template.serialize(to_str=False, qualified_save_path="sed_process.xml")
aiu_storage_template.serialize(to_str = False, qualified_save_path="aiu_template.xml")
aic_storage_template.serialize(to_str = False, qualified_save_path="aic_template.xml")
dip_storage_template.serialize(to_str = False, qualified_save_path="dip_template.xml")
# process_template.deploy(resolution="full", overwrite=True)

# process_template.validate()
process_template.visualize().show("sed_process_template.html")

sed_process_template.html


### Step 6: Trigger the process to produce data records

In [102]:
# python3 trigger_process.py -r low lapenta25_process_template nccf-sandbox-archive-workspace-us-east-1-188772587448 foldername/SED.csv sandbox

In [103]:
# sed_process_template = process_template.deploy(resolution="full", overwrite=True)

# print(f"Process = {process_template.node.iri}")
# print(f"AIU Template = {aiu_task_template.node.iri}")
# print(f"AIC Template = {aic_task_template.node.iri}")
# print(f"DIP Template = {dip_task_template.node.iri}")

# print(type(sed_process_template))
# print(sed_process_template)

# response = sed_process_template 

# for key, val in response.items():
#     print(f"iri: {key} type: {type(val)} value: {val}")
#     if isinstance(val, dict):
#         for subkey, subval in val.items():
#             print(f"  {subkey}: {subval}")

In [104]:
aiu_storage_template.deploy(overwrite=True)
aic_storage_template.deploy(overwrite=True)
dip_storage_template.deploy(overwrite=True)
process_template.deploy(resolution="low", overwrite=True)

/home/ec2-user/OISS/oiss-api/oiss/ontologies/base.py:483: UserWarning: This iri https://ncei.noaa.gov/oiss/template/storage/aiu/sed_aiu_template/sed_aiu_template already exists. Overwriting!!
  warnings.warn(f"This iri {self.node.iri} already exists. Overwriting!!")


No existing connection. Creating...


/home/ec2-user/OISS/oiss-api/oiss/ontology.py:255: UserWarning: This iri https://ncei.noaa.gov/oiss/template/storage/aic/sed_aic_template/sed_aic_template already exists. Overwriting!!
  warnings.warn(f"This iri {self.node.iri} already exists. Overwriting!!")
/home/ec2-user/OISS/oiss-api/oiss/ontology.py:269: UserWarning: OVERWRITE has been specified. If template exists in S3 or Neptune, specified template will be overwritten
  if overwrite: warnings.warn(f"OVERWRITE has been specified. If template exists in S3 or Neptune, specified template will be overwritten")
/home/ec2-user/OISS/oiss-api/oiss/ontologies/base.py:483: UserWarning: This iri https://ncei.noaa.gov/oiss/template/storage/dip/sed_dip_template/sed_dip_template already exists. Overwriting!!
  warnings.warn(f"This iri {self.node.iri} already exists. Overwriting!!")
/home/ec2-user/OISS/oiss-api/oiss/ontology.py:1474: UserWarning: This iri https://ncei.noaa.gov/oiss/template/process/lapenta25_process_template/lapenta25_process_

{'MD5OfMessageBody': 'b93dee6c248864fee25aeea72678a8b0',
 'MessageId': 'd2a48849-c01d-44d4-939b-6a7bcf92be23',
 'SequenceNumber': '18895766379047151616',
 'ResponseMetadata': {'RequestId': 'e7a3cd27-4181-52d8-8f9b-ac2293654afd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e7a3cd27-4181-52d8-8f9b-ac2293654afd',
   'date': 'Thu, 31 Jul 2025 20:23:00 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '146',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}